In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import datetime
from datetime import timedelta
from dateutil import rrule
import time

import plotly.graph_objects as go

plt.rcParams["font.family"] = "sans-serif"

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.despine(left=True)

<Figure size 640x480 with 0 Axes>

### Equity Data Uploading

In [171]:
MSCI = pd.read_csv('Historical_Data_Prices_Cleaned/MSCI.csv')
CMG = pd.read_csv('Historical_Data_Prices_Cleaned/CMG.csv')
DPZ = pd.read_csv('Historical_Data_Prices_Cleaned/DPZ.csv')
EL = pd.read_csv('Historical_Data_Prices_Cleaned/EL.csv')
META = pd.read_csv('Historical_Data_Prices_Cleaned/META.csv')

#### Data Integrity Testing

In [172]:
MSCI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          4113 non-null   object 
 1   Open          4113 non-null   float64
 2   High          4113 non-null   float64
 3   Low           4113 non-null   float64
 4   Close         4113 non-null   float64
 5   Volume        4113 non-null   int64  
 6   Dividends     4113 non-null   float64
 7   Stock Splits  4113 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 257.2+ KB


In [173]:
MSCI.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2007-11-15,20.722917,25.633348,19.875980,23.516006,11770500,0.0,0.0
1,2007-11-16,23.407877,23.407877,21.263508,22.488861,1242900,0.0,0.0
2,2007-11-19,22.263618,23.200655,22.101440,23.200655,469500,0.0,0.0
3,2007-11-20,22.903321,26.318095,22.759162,24.615213,522200,0.0,0.0
4,2007-11-21,24.326906,24.444035,23.371850,24.011557,167500,0.0,0.0


In [174]:
MSCI['Date'] = pd.to_datetime(MSCI['Date'])
CMG['Date'] = pd.to_datetime(CMG['Date'])
DPZ['Date'] = pd.to_datetime(DPZ['Date'])
EL['Date'] = pd.to_datetime(EL['Date'])
META['Date'] = pd.to_datetime(META['Date'])

In [175]:
MSCI.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2007-11-15,20.722917,25.633348,19.875980,23.516006,11770500,0.0,0.0
1,2007-11-16,23.407877,23.407877,21.263508,22.488861,1242900,0.0,0.0
2,2007-11-19,22.263618,23.200655,22.101440,23.200655,469500,0.0,0.0
3,2007-11-20,22.903321,26.318095,22.759162,24.615213,522200,0.0,0.0
4,2007-11-21,24.326906,24.444035,23.371850,24.011557,167500,0.0,0.0


In [176]:
MSCI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          4113 non-null   datetime64[ns]
 1   Open          4113 non-null   float64       
 2   High          4113 non-null   float64       
 3   Low           4113 non-null   float64       
 4   Close         4113 non-null   float64       
 5   Volume        4113 non-null   int64         
 6   Dividends     4113 non-null   float64       
 7   Stock Splits  4113 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 257.2 KB


In [177]:

# Simplify dataframes to keep only 'Date' and 'Close' columns, and rename 'Close' to the company's ticker for clarity
msci_df = MSCI[['Date', 'Close']].rename(columns={'Close': 'MSCI'})
cmg_df = CMG[['Date', 'Close']].rename(columns={'Close': 'CMG'})
dpz_df = DPZ[['Date', 'Close']].rename(columns={'Close': 'DPZ'})
el_df = EL[['Date', 'Close']].rename(columns={'Close': 'EL'})
meta_df = META[['Date', 'Close']].rename(columns={'Close': 'META'})

# Convert 'Date' to datetime format for all dataframes
msci_df['Date'] = pd.to_datetime(msci_df['Date'])
cmg_df['Date'] = pd.to_datetime(cmg_df['Date'])
dpz_df['Date'] = pd.to_datetime(dpz_df['Date'])
el_df['Date'] = pd.to_datetime(el_df['Date'])
meta_df['Date'] = pd.to_datetime(meta_df['Date'])

# Merge all dataframes on 'Date'
merged_df = msci_df.merge(cmg_df, on='Date', how='outer')\
                   .merge(dpz_df, on='Date', how='outer')\
                   .merge(el_df, on='Date', how='outer')\
                   .merge(meta_df, on='Date', how='outer')

# Sort by 'Date' to ensure chronological order
merged_df = merged_df.sort_values(by='Date').reset_index(drop=True)

merged_df.head()


,Date,MSCI,CMG,DPZ,EL,META
0,1995-11-17,NaN,NaN,NaN,6.460622,NaN
1,1995-11-20,NaN,NaN,NaN,6.226541,NaN
2,1995-11-21,NaN,NaN,NaN,6.109500,NaN
3,1995-11-22,NaN,NaN,NaN,6.296765,NaN
4,1995-11-24,NaN,NaN,NaN,6.530845,NaN


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'merged_pct_change' is your DataFrame from the previous step

# Create a Plotly figure
fig = go.Figure()

# Add traces for each stock
fig.add_trace(go.Scatter(x=merged_pct_change.index, y=merged_pct_change['CMG'], mode='lines', name='CMG', line=dict(color='royalblue')))
fig.add_trace(go.Scatter(x=merged_pct_change.index, y=merged_pct_change['DPZ'], mode='lines', name='DPZ', line=dict(color='firebrick')))
fig.add_trace(go.Scatter(x=merged_pct_change.index, y=merged_pct_change['EL'], mode='lines', name='EL', line=dict(color='forestgreen')))
fig.add_trace(go.Scatter(x=merged_pct_change.index, y=merged_pct_change['META'], mode='lines', name='META', line=dict(color='goldenrod')))

# Improve layout for a cleaner look
fig.update_layout(
    title='Historical Stock Prices (Percentage Change)',
    xaxis_title='Date',
    yaxis_title='Percentage Change',
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    yaxis=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.5)'
    )
)

# Show the figure
fig.show()
